In [1]:
# Setup stuff

import google.generativeai as genai

# api key stuff

import os
genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))

c:\Users\jobri\.virtualenvs\Kattis-1BYb5w23\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# model
# using gemini-pro
# use gemini-pro-vision for text and images
model = genai.GenerativeModel('gemini-pro')
prompt = ""
response = model.generate_content("What is Google?")

In [ ]:
response.text